# Clean csv generator

In [33]:
import plotly.graph_objects as go
import os
import pandas as pd
from plotly.subplots import make_subplots

In [34]:
# Dossiers
input_dir = './data/scopus'
output_dir = './data/cleanScopus'

overall_nb_pub = 0
overall_nb_journals = 0
overall_clean_journals = 0
overall_unclean_journals = 0

overall_clean_dict ={}
overall_unclean_dict = {}

for filename in os.listdir(input_dir):
    if filename.endswith('COMP.csv') and filename != "MEDI.csv.broken":
        print(filename)
        input_path = os.path.join(input_dir, filename)
        fig = go.Figure()

        nb_pub = 0
        nb_journals = 0
        nb_clean_journals = 0
        nb_unclean_journals =0
        clean_dict = {}
        unclean_dict = {}

        df = pd.read_csv(input_path)
        for ind, row in df.iterrows() :
            nb_pub += 1
            if row["prism:aggregationType"] == "Journal" :
                nb_journals += 1
                subtype = row["subtypeDescription"]
                issn = str(row["prism:issn"])
                e_issn = str(row["prism:eIssn"])
                if (issn and issn.strip() and issn.lower() != "nan") or  (e_issn and e_issn.strip() and e_issn.lower() != "nan") :
                    if subtype not in overall_clean_dict :
                        overall_clean_dict[subtype] = 0
                    if subtype not in clean_dict :
                        clean_dict[subtype] = 0
                    nb_clean_journals += 1
                    clean_dict[subtype] += 1
                    overall_clean_journals += 1
                else :
                    if subtype not in overall_unclean_dict :
                        overall_unclean_dict[subtype] = 0
                    if subtype not in unclean_dict :
                        unclean_dict[subtype] = 0
                    nb_unclean_journals += 1
                    unclean_dict[subtype] += 1
                    overall_unclean_journals += 1

        overall_nb_pub += nb_pub
        overall_nb_journals += nb_journals

        # output_path = os.path.join(output_dir, f"clean_{filename}")
        #
        # clean_journal_df.to_csv(output_path, index=False)

        # Créer un graphique en jauge pour afficher le pourcentage de journaux nettoyés
        pub_percentage = nb_clean_journals / nb_pub * 100
        journal_percentage = nb_clean_journals / nb_journals * 100

        fig.add_trace(go.Indicator(
            value=pub_percentage,
            gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#4CAF50"}},
            title={'text': filename + " % sur les publications"},
            domain={'row': 0, 'column': 0}))
        fig.add_trace(go.Indicator(
            value=journal_percentage,
            gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#4CAF50"}},
            title={'text': filename + " % sur les journaux"},
            domain={'row': 0, 'column': 1}))
        # fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
        if "Article" not in clean_dict:
            clean_dict["Article"] = 0
        if "Article" not in unclean_dict:
            unclean_dict["Article"] = 0

        clean_articles = clean_dict["Article"]
        unclean_articles = unclean_dict["Article"]
        fig.add_trace(go.Indicator(
            value=sum([value for (key,value) in clean_dict.items() if key!="Article"])/(nb_journals - clean_articles - unclean_articles)*100,
            gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#4CAF50"}},
            title={'text': filename + " % Journaux non Articles propres"},
            domain={'row': 1, 'column': 0}))
        fig.add_trace(go.Indicator(
            value=clean_articles/(clean_articles + unclean_articles)*100,
            gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#4CAF50"}},
            title={'text': filename + " % Journaux Articles propres"},
            domain={'row': 1, 'column': 1}))
        fig.update_layout(
            grid={'rows': 2, 'columns': 2, 'pattern': "independent"},
            template={'data': {
                'indicator': [{
                    'title': {'text': filename},
                    'mode': "number+gauge",
                    'delta': {'reference': 90}}]
                }
            }
        )
        fig1 = go.Figure(data = [go.Pie(
            labels = list(clean_dict.keys()),
            values = list(clean_dict.values()),
            title={'text': filename + " Distrib subtype/clean Journals"})])
        fig2  = go.Figure(data = [go.Pie(
            labels = list(unclean_dict.keys()),
            values = list(unclean_dict.values()),
            title={'text': filename + " Distrib subtype/unclean Journals"})])
        # fig.write_image(output_dir + "/charts/" + filename + ".png")
        fig.show()
        fig1.show()
        fig2.show()
        break
        # print("Pourcentage pour publis :" + str(pub_percentage))
        # print("Pourcentage pour journaux :" + str(journal_percentage))

overall_percentage_pub = overall_clean_journals / overall_nb_pub * 100
overall_percentage_journal = overall_clean_journals / overall_nb_journals * 100

fig1 = go.Figure()
fig1.add_trace(go.Indicator(
    mode="gauge+number",
    value=overall_percentage_pub,
    title={'text': "Pourcentage pour les publications"},
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#4CAF50"}},
    domain={'row': 0, 'column': 0}
))
fig1.add_trace(go.Indicator(
    mode="gauge+number",
    value=overall_percentage_journal,
    title={'text': "Pourcentage pour les journaux"},
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#4CAF50"}},
    domain={'row': 0, 'column': 1}
))
fig1.update_layout(
            grid={'rows': 1, 'columns': 2, 'pattern': "independent"},
            template={'data': {
                'indicator': [{
                    'mode': "number+gauge",
                    'delta': {'reference': 120}}]
                }
            }
        )
fig1.show()
# Sauvegarder la figure en format PNG
# fig.write_image(output_dir + "gauge_charts/overall_journal.png")

print("Traitement terminé. Les fichiers nettoyés sont disponibles dans le dossier ./data/cleanScopus.")



COMP.csv


Traitement terminé. Les fichiers nettoyés sont disponibles dans le dossier ./data/cleanScopus.


In [35]:
print(unclean_dict)

{'Article': 116, 'Review': 11, 'Editorial': 1, 'Conference Paper': 1}
